---
title: "Simon problem"
author: "Mi-Song Dupuy"
date: "2025-07-02"
format:
  html:
    code-fold: true
jupyter: python3.12
---

In the Simon problem, a function $f : \lbrace 0,1 \rbrace^n \to \lbrace 0,1 \rbrace^n$ is given which has the property 
$$
\exists s \in \lbrace 0,1 \rbrace^n, \forall\, x,y \in \lbrace 0,1 \rbrace^n, f(x) = f(y) \Rightarrow x = y \oplus s,
$$
where $\oplus$ is the addition (modulo 2) in $\lbrace 0,1 \rbrace^n$. 

The goal is to determine the vector $s$.

**Preliminary question**

1. Show that if $s \not= 0$, then there is exactly a pair $(x,y) \in \lbrace 0,1 \rbrace^n$ such that $f(x) = f(y)$. 

*Solution:* If $f(x)=f(y)$, then by property of $f$, then $x=y \oplus s$. Thus $x \oplus s = y \oplus s\oplus s =y$, hence there is exactly one pair $(x,y) \in \lbrace 0,1 \rbrace^n$ that satisfies $f(x)=f(y)$.

In the Simon problem, we thus work only with functions $f$ that are either 1-to-1 (if $s =0$) or 2-to-1.

# Mathematical formulation of the quantum algorithm

In this tutorial, we will demonstrate the quantum advantage for this problem and implement the solution using MyQLM. 

1. Show that classically, the cost to determine $s$ is of the order $2^{n-1}$ in the worst case scenario.

*Solution:* 

- if $s=0$, then to check that the function is indeed 1-to-1 and not 2-to-1, at least $2^{n-1}+1$ values need to be computed.
- if $s \not= 0$, then in the worst case scenario $2^{n-1}+1$ values need to be computed in order to find a pair such that $f(x)=f(y)$. 

Let $U_f$ be a quantum gate acting on $2n$ qubits such that for $|x\rangle, |w\rangle$ two $n$-qubit states, we have
$$
U_f (|x, w\rangle) = U_f ( |x\rangle \otimes |w\rangle ) = |x\rangle \otimes |f(x)\oplus w\rangle.
$$
In this setting, $U_f$ is called an *oracle*.

2. Check that $U_f^{-1} = U_f$ and deduce that it indeed defines a quantum gate.

*Solution:* by definition of $U_f$, we have that 
$$
U_f^2 |x,w \rangle = U_f(| x, f(x) \oplus w \rangle) = | x, f(x) \oplus f(x) \oplus w \rangle = |x, w\rangle.
$$

We consider the following circuit to determine the period $s$.

![Simon circuit](simon_circuit.png)

3. We suppose that the output of the measure of the last $n$ qubits give $|f(x)\rangle$ for some quantum state $x$. Show that the output $|y\rangle$ of the first $n$ qubits is such that $y \perp s$ (in the modulus 2 sense).

*Solution*: 

- if $s=0$, any state $y \perp 0$;
- if $s \not=0$, then we have after $U_f\big( H^{\otimes n}(\vert0\rangle^{\otimes n}) \otimes \vert0\rangle^{\otimes n}\big) = \frac{1}{2^{n-1}}U_f\big(\sum_{j=0}^{2^{n}-1} |j \rangle \otimes \vert0\rangle^{\otimes n} \big) = \sum_{j=0}^{2^n-1} \vert j\rangle \oplus \vert f(j) \rangle$. After the measurement, the sum collapses to a value $f(x)$, for some $x \in \lbrace 0, 1 \rbrace^n$. Since the function is 2-to-1, there are exactly two values $x,\tilde{x}=x \oplus s \in \lbrace 0,1 \rbrace^n$, such that $f(x)=f(\tilde{x})$. Thus the measurement gives on the first register $\frac{\vert x \rangle + \vert \tilde{x} \rangle}{\sqrt{2}}$. Thus we have 
$$
H^{\otimes n} \big(\vert x \rangle + \vert x \oplus s \rangle \big) = \sum_{j=0}^{2^n-1} (-1)^{j \cdot x} \vert j\rangle + \sum_{j=0}^{2^n-1} (-1)^{j \cdot (x + s) } \vert j\rangle = \sum_{j=0}^{2^n-1} (-1)^{j \cdot x} (1 + (-1)^{j \cdot s}) \vert j\rangle,
$$
where we identify $j$ with its binary representation and $\cdot$ denotes the Euclidian scalar product. 
Hence for $j \in \lbrace 0, \dots, 2^n-1 \rbrace$, if $j \cdot s = 1\  \mathrm{mod} \ 2$, then $1 + (-1)^{j \cdot s} = 0$, thus only the terms such that $j \cdot s = 0\  \mathrm{mod} \ 2$ remain in the sum. 

4. Suppose that the circuit above is run $n+k$ times (for $k \in \mathbb{N}\setminus \lbrace 0 \rbrace$) and the result of the first $n$ qubits are stored in the vectors $(y_i)_{1\leq i \leq n+k}$. Show that with probability $1-\frac{1}{2^{k+1}}$, $\mathrm{Span}(y_1,\dots,y_{n+k}) = s^{\perp}$.

*Solution*: $n+k$ runs of the Simon circuit yields $n+k$ vectors $(y_i)_{1 \leq i \leq n+k}$ such that $y \perp s$. Let us assume that we have $m$ linearly independent vectors $(z_i)_{1\leq i \leq m}$ where $z_i$ is such that $(z_i)_j = 0$ if $j \not= i$ and $(z_i)_i = 1$ otherwise. Then the probability that $z_{m+1} \in \lbrace 0,1 \rbrace^n$ depends linearly of $(z_i)_{1 \leq i \leq m}$ is $\frac{2^m}{2^n} = \frac{1}{2^{n-m}}$. Then if $s \not= 0$, since the events $A_m = \lbrace s,(y_i)_{1 \leq i \leq n+k} \text{ has exactly } m \text{ independent vectors} \rbrace$ are independent for $m \in \lbrace 1, \dots, n-1\rbrace$, we have that the probability that the vectors $s, (y_i)_{1 \leq i \leq n+k}$ are linearly dependent is equal to 
$$
\mathbb{P}(s, (y_i)_{1 \leq i \leq n+k} \text{ linearly dependent}) = \sum_{m=1}^{n-1} \underbrace{\mathbb{P}(A_m)}_{= (\frac{2^m}{2^n})^{n-m+k+1}} = \sum_{m=1}^{n-1} \Big( \frac{1}{2^m}\Big)^{m+k+1} \leq \frac{1}{2^{k+1}}.
$$


5. Prove that the quantum algorithm exhibits a quantum advantage.

*Solution:* the linear system can be solved in polynomial complexity with respect to $n$, hence this quantum algorithm exhibits an exponential speed-up.

# Implementation

For $k \in \lbrace 0,\dots,n-1 \rbrace$, we consider functions $f : \lbrace 0,1 \rbrace^n \to \lbrace 0,1 \rbrace^n$ of the form: for all $x = (x_1,\dots,x_n) \in \lbrace 0,1 \rbrace^n$
$$
f(x_1,\dots,x_n) = (x_1,\dots,x_{k-1},0,x_{k+1},\dots,x_n).
$$

1. Implement the oracle $U_f$ given $s\in \lbrace 0,1 \rbrace^n$.

In [ ]:
from qat.lang.AQASM import QRoutine, X, H, CNOT

def hadamard_generator(n):
    routine = QRoutine() #QRoutine creates subcircuits that behave as a Gate object.
    for i in range(n):
        routine.apply(H,i)
    return routine

def Uf_generator(n,k):
  Uroutine = QRoutine()
  Uroutine.new_wires(2*n)
  for i in range(n):
    if i==k:
      Uroutine.apply(X,n+k-1)
    else:
      Uroutine.apply(CNOT,i,n+i)
  return Uroutine

2. Implement the Simon quantum circuit.

In [ ]:
from qat.lang.AQASM import Program

n = 2
k = 0
simon = Program()
qubits = simon.qalloc(2*n)

Hs = hadamard_generator(n)
Uf = Uf_generator(n,k)

Hs(qubits[0:n])
Uf(qubits)
simon.measure(range(n,2*n))
simon.apply(Hs,qubits[0:n])

simon_circ = simon.to_circ()
simon_circ.display()

3. Test for $n=2, k=0$ and check that the output is indeed orthogonal to $\vert10\rangle$.

In [ ]:
from qat.qpus import PyLinalg 
import numpy as np

linalgqpu = PyLinalg()

k=5
simon_job = simon_circ.to_job(qubits=range(n),nbshots=n+k)

Y = np.zeros((n,n+k))
result = linalgqpu.submit(simon_job)
i = 0
for sample in result:
    for j in range(n):
      Y[j,i] = np.array(sample.state[j])
    print("We measured the state {} (its probability is {} and its amplitude {})".format(sample.state, sample.probability, sample.amplitude)) #the amplitude is not returned
    i += 1

We see that the output is indeed orthogonal to $\vert10\rangle$.